<a href="https://colab.research.google.com/github/Selma266/Malicious-URL-Detection/blob/main/Malicious_URL_Detection_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Chargement Phishing URLs dataset + prétraitement

In [2]:
import pandas as pd #la bibliothèque Pandas, utilisée pour la manipulation et l’analyse de données tabulaires.
df = pd.read_csv("/content/drive/MyDrive/Phishing URLs_cleaned.csv")  # read dataset url,label  #Chargement du dataset
df.head()

,url,label
0,https://docs.google.com/presentation/d/e/2PACX...,malicious
1,https://btttelecommunniccatiion.weeblysite.com/,malicious
2,https://kq0hgp.webwave.dev/,malicious
3,https://brittishtele1bt-69836.getresponsesite....,malicious
4,https://bt-internet-105056.weeblysite.com/,malicious


Suppression des lignes redondantes

In [3]:
# BEFORE removing duplicates
print("Before removing duplicates:")
print("Shape:", df.shape)

# Step 2: Remove duplicate rows
df = df.drop_duplicates()   #Suppression des doublons

# AFTER removing duplicates
print("\nAfter removing duplicates:")
print("Shape:", df.shape)

Before removing duplicates:
Shape: (504983, 2)

After removing duplicates:
Shape: (504933, 2)


Supprimer les NaN+ Conversion label en valeur Numeriques

In [4]:
# Supprimer les lignes où url ou label est NaN
df = df.dropna(subset=["url", "label"]).reset_index(drop=True)  #reset_index(drop=True) Réinitialise l’index du DataFrame
# strip() supprime les espaces au début et à la fin URL
df["url"] = df["url"].str.strip()
# Conversion des labels texte en valeurs numériques
df["label"] = df["label"].map({"benign": 0, "malicious": 1}).astype(int) #Encodage des labels
print(df["label"].value_counts()) #Vérification de la distribution des classes Afichages

label
0    345738
1    159195
Name: count, dtype: int64
